In [1]:
!pip install dagshub mlflow --quiet

import warnings
warnings.filterwarnings("ignore")

print("Done!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 7.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 54.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 46.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 733.8/733.8 kB 20.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.5/95.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are install

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/walmart-recruiting-store-sales-forecasting/train.csv.zip
/kaggle/input/walmart-recruiting-store-sales-forecasting/sampleSubmission.csv.zip
/kaggle/input/walmart-recruiting-store-sales-forecasting/stores.csv
/kaggle/input/walmart-recruiting-store-sales-forecasting/features.csv.zip
/kaggle/input/walmart-recruiting-store-sales-forecasting/test.csv.zip


In [5]:
import mlflow.sklearn
from datetime import datetime
import joblib
import dagshub
import mlflow
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from tqdm import tqdm
from sklearn.compose import ColumnTransformer


dagshub.init(repo_owner='gnada22', repo_name='ml_final_project', mlflow=True)

Initialized MLflow to track repo "gnada22/ml_final_project"

Repository gnada22/ml_final_project initialized!

In [35]:
# class definitions

class DateFeatureCreator(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X["week"] = (
            X["Date"].dt.to_period("W")
            .rank(method="dense")
            .astype(int) - 1
        )
        X["sin_13"] = np.sin(2 * np.pi * X["week"] / 13)
        X["cos_13"] = np.cos(2 * np.pi * X["week"] / 13)
        X["sin_23"] = np.sin(2 * np.pi * X["week"] / 23)
        X["cos_23"] = np.cos(2 * np.pi * X["week"] / 23)
        X = X.drop(columns=["Date"])
        return X

date_features = ["week", "sin_13", "cos_13", "sin_23", "cos_23"]

class LagFeatureAdder:
    def transform(self, df: pd.DataFrame) -> pd.DataFrame:
        df = df.sort_values(["Store", "Dept", "Date"])
        df["lag_1"] = df.groupby(["Store", "Dept"])["Weekly_Sales"].shift(1)
        df["lag_52"] = df.groupby(["Store", "Dept"])["Weekly_Sales"].shift(52)
        return df

lag_features = ["lag_1", "lag_52"]

added_features = date_features + lag_features

class ColumnTransformerWithNames(ColumnTransformer):
    def get_feature_names_out(self, input_features=None):
        return super().get_feature_names_out(input_features)

    def transform(self, X):
        X_transformed = super().transform(X)
        # Get feature names for columns
        cols = self.get_feature_names_out()
        cols = [c.split("__", 1)[-1] for c in self.get_feature_names_out()]
        res = pd.DataFrame(X_transformed, columns=cols, index=X.index)
        # print("with name transform - ", type(res))
        return res

    def fit_transform(self, X, y=None):
        X_transformed = super().fit_transform(X, y)
        cols = self.get_feature_names_out()
        cols = [c.split("__", 1)[-1] for c in self.get_feature_names_out()]
        res = pd.DataFrame(X_transformed, columns=cols, index=X.index)
        # print("with name fit_transform - ", type(res))
        return res

def extract_onehot_value(row, prefix, default=1):
    for col in row.index:
        if col.startswith(prefix) and row[col] == 1:
            return int(col.split("_")[1])
    return default

class ResidualRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, base_model, residual_model):
        self.base_model = base_model
        self.residual_model = residual_model

    def fit(self, X, y):
        self.base_model_ = clone(self.base_model)
        self.base_model_.fit(X, y)
        residuals = y - self.base_model_.predict(X)
        
        self.residual_model_ = clone(self.residual_model)
        self.residual_model_.fit(X, residuals)
        return self

    def extract_onehot_value(self, row, prefix, default=1):
        for col in row.index:
            if col.startswith(prefix) and row[col] == 1:
                return int(col.split("_")[1])
        return default
    
    def predict(self, X):
        pred_lookup = {}
        weekly_preds = pd.Series(index=X.index, dtype=float)
        
        for week in tqdm(X["week"].sort_values().unique(), desc="Recursive prediction"):
            day_rows = X[X["week"] == week].copy()
        
            for idx, row in day_rows.iterrows():
                store = extract_onehot_value(row, "Store_")
                dept = extract_onehot_value(row, "Dept_")
        
                # Get keys for previous lags
                key_1 = (store, dept, week - 1)
                key_52 = (store, dept, week - 52)
        
                lag_1 = pred_lookup.get(key_1, row["lag_1"])
                lag_52 = pred_lookup.get(key_52, row["lag_52"])
        
                day_rows.at[idx, "lag_1"] = lag_1
                day_rows.at[idx, "lag_52"] = lag_52
        
            # Predict all rows for this day in one batch
            y_preds = self.pred_f(day_rows)
        
            # Assign predictions back
            weekly_preds[day_rows.index] = y_preds
            
            # Update lookup for future lag access
            for idx, pred in zip(day_rows.index, y_preds):
                row = day_rows.loc[idx]
                
                store = extract_onehot_value(row, "Store_")
                dept = extract_onehot_value(row, "Dept_")
                
                key = (store, dept, row["week"])
                pred_lookup[key] = pred
    
        return weekly_preds.fillna(0).to_numpy()

    def pred_f(self, X):
        return self.base_model_.predict(X) + self.residual_model_.predict(X)

In [36]:
local_path = mlflow.artifacts.download_artifacts(
    artifact_uri="mlflow-artifacts:/62e2117acafa4300989c6ce37ac3c340/61e2644fb47f4ef69d86fd860e32dc6c/artifacts/model.pkl"
)

model = joblib.load(local_path)

print("Done!")

Done!


In [8]:
# load and add lag features

test = pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/test.csv.zip", parse_dates=["Date"])
features = pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/features.csv.zip", parse_dates=["Date"])
stores = pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/stores.csv")

df = test.merge(features, on=["Store", "Date", "IsHoliday"], how="left")
df = df.merge(stores, on="Store", how="left")

def add_lag_features(df):
    train = pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/train.csv.zip", parse_dates=["Date"])
    train = train[["Store", "Dept", "Date", "Weekly_Sales"]]
    train = train.sort_values(["Store", "Dept", "Date"])
    
    full = pd.concat([train, df], axis=0)
    full = full.sort_values(["Store", "Dept", "Date"])
    
    full["lag_1"] = full.groupby(["Store", "Dept"])["Weekly_Sales"].shift(1)
    full["lag_52"] = full.groupby(["Store", "Dept"])["Weekly_Sales"].shift(52)

    res = full[full["Weekly_Sales"].isna()].copy() # just test rows
    # res.dropna(subset=["lag_1", "lag_52"], inplace=True)  # drop rows with missing lags

    return res

df = add_lag_features(df)

X_test = df.drop(columns=["Weekly_Sales"], errors="ignore")

print("Done!")

Done!


In [37]:
# X_test = X_test.sort_values(["Date", "Store", "Dept"]).reset_index(drop=True)

preds = model.predict(X_test)

submission = pd.DataFrame()
submission["Weekly_Sales"] = preds
submission["Id"] = X_test["Store"].astype(str) + "_" + X_test["Dept"].astype(str) + "_" + X_test["Date"].dt.strftime("%Y-%m-%d")
# print(submission)
submission.to_csv("submission.csv", index=False)
print("✅ Submission saved as submission.csv")

Recursive prediction: 100%|██████████| 39/39 [00:45<00:00,  1.18s/it]


✅ Submission saved as submission.csv
